In [2]:
import findspark
findspark.init('D:/spark-2.2.0-bin-hadoop2.7')

import io
import re
import os.path as pth
import itertools
import zipfile as zf
import pandas as pd
import urllib.request as ureq
import datetime as dt

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

### Get weatherstations in/around Stuttgart

In [44]:
lat_bounds = (48.3, 49.1)
lon_bounds = (8.9, 9.5)
#stat_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/air_temperature/historical/TU_Stundenwerte_Beschreibung_Stationen.txt"
stat_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/air_temperature/recent/TU_Stundenwerte_Beschreibung_Stationen.txt"
col_names = ["Stations_id", "von_datum",  "bis_datum" , "Stationshoehe", "geoBreite", "geoLaenge", "Stationsname", "Bundesland"]
df = pd.read_fwf(stat_url, skiprows=2, names=col_names, delim_whitespace=True, error_bad_lines=False, encoding="iso-8859-1")

In [47]:
stgt_stats_df = df[(df["geoBreite"] > lat_bounds[0]) & (df["geoBreite"] < lat_bounds[1]) & 
               (df["geoLaenge"] > lon_bounds[0]) & (df["geoLaenge"] < lon_bounds[1])]
stgt_stats_df

,Stations_id,von_datum,bis_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland
194,2074,20040601,20180106,522,48.3751,8.9801,Hechingen,Baden-Württemberg
315,3278,20090401,20180106,355,48.5376,9.2733,Metzingen,Baden-Württemberg
330,3402,20021101,20180106,750,48.3851,9.4837,Münsingen-Apfelstetten,Baden-Württemberg
359,3671,19790101,19810401,280,48.6321,9.3261,Nürtingen,Baden-Württemberg
391,4160,20040701,20180106,478,48.7425,8.9240,Renningen-Ihinger Hof,Baden-Württemberg
402,4294,19820101,19970101,360,48.4704,8.9687,Rottenburg-Kiebingen,Baden-Württemberg
409,4349,20040601,20180106,248,48.9569,9.0710,Sachsenheim,Baden-Württemberg
450,4926,19790401,20130514,224,48.7896,9.2167,Stuttgart (Neckartal),Baden-Württemberg
451,4927,19760101,19840801,286,48.7693,9.1814,Stuttgart-Stadt,Baden-Württemberg
452,4928,19770701,20180106,314,48.8282,9.2000,Stuttgart (Schnarrenberg),Baden-Württemberg


In [172]:
class DWDBase(object):
    def __init__(self, station_ids):
        self.station_ids = station_ids
        self.pattrn = '(?:stundenwerte\_(\d|\w||_)+\.zip)'
        
    def download_products(self):
        all_urls = (self.infer_urls_for_stations(self.hist_base_url, self.station_ids) + 
                     self.infer_urls_for_stations(self.rece_base_url, self.station_ids))
        all_dfs = [df for df in [self.download_products_from_url(u) for u in all_urls] if df is not None]
        df = pd.concat(all_dfs)
        return self.postprocess_data(df)
            
    def infer_urls_for_stations(self, base_url, station_ids):
        with ureq.urlopen(base_url) as res:
            listing = [re.search(self.pattrn, f).group(0) for f in [p for p in 
                   [l.decode("utf-8") for l in res.readlines()] 
                   if re.search(self.pattrn, p)
            ]]
        files = [[l for l in listing if ("_%05d_" % d) in l] for d in station_ids]
        return [pth.join(base_url, f) for f in itertools.chain(*files)]
    
    def download_products_from_url(self, url):
        try:
            with ureq.urlopen(url) as resp:
                with zf.ZipFile(io.BytesIO(resp.read())) as z:
                    products = [f.filename for f in z.filelist if "produkt_" in f.filename]
                    if not len(products):
                        raise Exception("No products in zip")

                    with z.open(products[0]) as f:
                        df = pd.read_csv(f, header=0, sep=";")
                        return df
        except Exception as ex:
            print("Unable to extract product from '%s' due to %s" % (url, ex))
            return None
        
    def postprocess_data(self, df):
        if "eor" in df.columns:
            df = df.drop('eor', 1)
        
        if "MESS_DATUM" in df.columns:
            df["TIMESTAMP"] = pd.to_datetime(df["MESS_DATUM"], format="%Y%m%d%H")
        
        return df

class DWDTemp(DWDBase):
    def __init__(self, station_ids):
        super().__init__(station_ids)
        self.hist_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/air_temperature/historical/"
        self.rece_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/air_temperature/recent/"
        

class DWDPrecipitation(DWDBase):
    def __init__(self, station_ids):
        super().__init__(station_ids)
        self.hist_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/precipitation/historical/"
        self.rece_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/precipitation/recent/"
        
class DWDCloudiness(DWDBase):
    def __init__(self, station_ids):
        super().__init__(station_ids)
        self.hist_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/cloudiness/historical/"
        self.rece_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/cloudiness/recent/"
        
class DWDSun(DWDBase):
    def __init__(self, station_ids):
        super().__init__(station_ids)
        self.hist_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/sun/historical/"
        self.rece_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/sun/recent/"
        
class DWDWind(DWDBase):
    def __init__(self, station_ids):
        super().__init__(station_ids)
        self.hist_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/wind/historical/"
        self.rece_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/wind/recent/"

In [189]:
def join_station_data(ts_df, stations_df):
    right_df = stations_df.drop("von_datum", 1) \
                          .drop("bis_datum", 1) \
                          .drop("Bundesland", 1)  
    return pd.merge(left=ts_df, right=right_df, left_on="STATIONS_ID", right_on="Stations_id") \
             .drop("Stations_id", 1)

### Temperatur
- QN_9 - Qualitätsniveau der nachfolgenden Spalten
- TT_TU - Lufttemperatur in 2m Höhe in °C
- RF_TU - relative Feuchte in % 

#### Qualitaetsinformation
Die hier abgegebenen Qualitätsniveaus gelten jeweils für die nachfolgenden Spalten. Das Qualitätsniveau
(QN) beschreibt das Verfahren der Qualitätsprüfung und bezieht sich auf einen vollständigen Satz von
Parametern zu einem bestimmten Termin. Die einzelnen Parameter eines vollständigen Satzes sind
in der internen DWD-Datenbank mit jeweiligen Qualitätsbytes verknüpft, die hier nicht mit ausgegeben
werden. Als falsch oder zweifelhaft markierte Werte sind hier auf -999 gesetzt worden. Verschiedene
Qualitätsprüfverfahren (auf verschiedenen Stufen) entscheiden, welche Werte falsch oder zweifelhaft sind. In
der Vergangenheit wurden zum Teil andere Verfahren benutzt.
Qualitätsniveau (z.B. QN_9)

- 1 - nur formale Prüfung
- 2 - nach individuellen Kriterien geprüft
- 3 - alte automatische Prüfung und Korrektur
- 5 - historische, subjektive Verfahren
- 7 - 2. Prüfung durchlaufen, vor Korrektur
- 8 - Qualitätsicherung ausserhalb ROUTINE
- 9 - nicht alle Parameter korrigiert
- 10 - Qualitätsprüfung abgeschlossen, Korrekturen beendet

In [191]:
temp = DWDTemp(stgt_stats_df["Stations_id"].values)
temp_df = join_station_data(temp.download_products(), stgt_stats_df)

In [216]:
temp_df.sample(n=10)

,STATIONS_ID,MESS_DATUM,QN_9,TT_TU,RF_TU,TIMESTAMP,Stationshoehe,geoBreite,geoLaenge,Stationsname
788133,4926,2007070805,7,17.0,71.0,2007-07-08 05:00:00,224,48.7896,9.2167,Stuttgart (Neckartal)
753348,4926,2003071717,10,22.4,60.0,2003-07-17 17:00:00,224,48.7896,9.2167,Stuttgart (Neckartal)
105938,2074,2016080211,7,20.6,60.0,2016-08-02 11:00:00,522,48.3751,8.9801,Hechingen
1348468,4931,1996072604,10,14.6,87.0,1996-07-26 04:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
104338,2074,2016052719,7,18.8,75.0,2016-05-27 19:00:00,522,48.3751,8.9801,Hechingen
789701,4926,2007091113,7,15.6,55.0,2007-09-11 13:00:00,224,48.7896,9.2167,Stuttgart (Neckartal)
1372990,4931,1999051322,10,13.2,80.0,1999-05-13 22:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
79265,2074,2013071720,7,21.1,62.0,2013-07-17 20:00:00,522,48.3751,8.9801,Hechingen
1737122,4933,1973052508,5,12.2,78.0,1973-05-25 08:00:00,401,48.7091,9.2147,Stuttgart-Hohenheim
1621238,4933,1960030520,5,5.1,84.0,1960-03-05 20:00:00,401,48.7091,9.2147,Stuttgart-Hohenheim


### Niederschlagshöhe
- QN_8 - Qualitätsniveau der nachfolgenden Spalten
- R1 stündliche Niederschlagshöhe in mm
- RS_IND - Indikator ob Niederschlag gefallen (0 = kein Niederschlag, 1 = Niederschlag)
- WRTR - WR-Niederschlagsform (siehe Unten)

#### WR-Codierung
- -999  - Fehlwerte sind mit -999 gekennzeichnet
- 0 - kein Niederschlag gefallenen und/ oder keine Niederschlagshöhe aus abgesetzten Niederschlägen (wie Tau, Reif); 
- 1 - Niederschlagshöhe ausschließlich aus abgesetzten Niederschlägen (fest und flüssig) oder es kann nicht zwischen fest und flüssig unterschieden werden; 
- 2 - Niederschlagshöhe ausschließlich aus flüssigen abgesetzten Niederschlägen ("weisser Tau" wird den flüssigen abgesetzten Niederschlägen zugeordnet) ; 
- 3 - Niederschlagshöhe ausschließlich aus festen abgesetzten Niederschlägen; 
- 6 - gefallener Niederschlag nur in flüssiger Form, kann auch abgesetzten Niederschlag jeder Art enthalten; 
- 7 - gefallener Niederschlag nur in fester Form, kann auch abgesetzten Niederschlag jeder Art enthalten; 
- 8 - gefallener Niederschlag in fester und flüssiger Form, kann auch abgesetzten Niederschlag jeder Art enthalten;
- 9 - Niederschlagsmessung ausgefallen, die Niederschlagsform kann nicht festgestellt werden.

In [193]:
pert = DWDPrecipitation(stgt_stats_df["Stations_id"].values)
pert_df = join_station_data(pert.download_products(), stgt_stats_df)

In [218]:
pert_df.sample(10)

,STATIONS_ID,MESS_DATUM,QN_8,R1,RS_IND,WRTR,TIMESTAMP,Stationshoehe,geoBreite,geoLaenge,Stationsname
173279,3278,2009080608,3,0.0,0,0,2009-08-06 08:00:00,355,48.5376,9.2733,Metzingen
1053849,4931,1999100317,1,0.0,0,-999,1999-10-03 17:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
162858,3278,2007080822,3,0.6,1,6,2007-08-08 22:00:00,355,48.5376,9.2733,Metzingen
1241670,6275,2007030908,3,0.0,0,0,2007-03-09 08:00:00,325,48.6705,9.4627,Notzingen
1177555,4931,2013111718,3,0.0,0,-999,2013-11-17 18:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
703958,4349,2012102606,3,0.0,0,-999,2012-10-26 06:00:00,248,48.9569,9.0710,Sachsenheim
278273,3402,2006011514,3,0.0,0,0,2006-01-15 14:00:00,750,48.3851,9.4837,Münsingen-Apfelstetten
298108,3402,2008042101,3,0.0,0,0,2008-04-21 01:00:00,750,48.3851,9.4837,Münsingen-Apfelstetten
463716,4160,2013031003,3,0.0,0,-999,2013-03-10 03:00:00,478,48.7425,8.9240,Renningen-Ihinger Hof
319106,3402,2010091301,3,1.6,1,6,2010-09-13 01:00:00,750,48.3851,9.4837,Münsingen-Apfelstetten


### Wolkenbedeckung
- QN_9 - Qualitätsniveau der nachfolgenden Spalten (siehe Temperatur)
- V_N_I - Index wie Messung erhoben wurde (P = vom Beobachter, I = vom Instrument)
- V_N - Bedeckungsgrad aller Wolken auf Skala 1-8 (nicht erkennbar -1)

In [195]:
cloud = DWDCloudiness(stgt_stats_df["Stations_id"].values)
cloud_df = join_station_data(cloud.download_products(), stgt_stats_df)

In [217]:
cloud_df.sample(10)

,STATIONS_ID,MESS_DATUM,QN_8,V_N_I,V_N,TIMESTAMP,Stationshoehe,geoBreite,geoLaenge,Stationsname
193503,4928,2012121617,3,I,8,2012-12-16 17:00:00,314,48.8282,9.2000,Stuttgart (Schnarrenberg)
479619,4931,1982102709,1,P,7,1982-10-27 09:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
364359,4931,1962122312,1,P,2,1962-12-23 12:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
275346,4931,1952102714,1,P,1,1952-10-27 14:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
683214,4931,2006022521,3,P,8,2006-02-25 21:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
36464,4927,1958080809,1,P,7,1958-08-08 09:00:00,286,48.7693,9.1814,Stuttgart-Stadt
626020,4931,1999081716,1,P,7,1999-08-17 16:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
677577,4931,2005070600,3,P,4,2005-07-06 00:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
310326,4931,1956102403,1,P,0,1956-10-24 03:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
601436,4931,1996102503,1,P,3,1996-10-25 03:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen


### Sonnenscheindauer
- QN_7 - Qualitätsniveau der nachfolgenden Spalten (siehe Temperatur)
- SD_SO - Stündliche Sonnenscheindauer

In [197]:
sun = DWDSun(stgt_stats_df["Stations_id"].values)
sun_df = join_station_data(sun.download_products(), stgt_stats_df)

In [219]:
sun_df.sample(10)

,STATIONS_ID,MESS_DATUM,QN_7,SD_SO,TIMESTAMP,Stationshoehe,geoBreite,geoLaenge,Stationsname
711106,4926,1954072717,5,0.0,1954-07-27 17:00:00,224,48.7896,9.2167,Stuttgart (Neckartal)
1276129,4931,2004022506,10,0.0,2004-02-25 06:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
1053338,4928,1993011615,7,30.0,1993-01-16 15:00:00,314,48.8282,9.2000,Stuttgart (Schnarrenberg)
201762,3402,1953122213,5,0.0,1953-12-22 13:00:00,750,48.3851,9.4837,Münsingen-Apfelstetten
513291,4160,2005060804,10,0.0,2005-06-08 04:00:00,478,48.7425,8.9240,Renningen-Ihinger Hof
725167,4926,1956091520,5,0.0,1956-09-15 20:00:00,224,48.7896,9.2167,Stuttgart (Neckartal)
708326,4926,1954022309,5,12.0,1954-02-23 09:00:00,224,48.7896,9.2167,Stuttgart (Neckartal)
1438625,4933,1961042315,5,0.0,1961-04-23 15:00:00,401,48.7091,9.2147,Stuttgart-Hohenheim
1298333,4931,2007071516,10,55.0,2007-07-15 16:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
1280149,4931,2004100512,10,60.0,2004-10-05 12:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen


### Windgeschwindigkeit und Windrichtung
- QN_3 - Qualitätsniveau der nachfolgenden Spalten (siehe Temperatur)
- F - mittlere Windgeschwindigkeit in m/s
- D - mittlere Windrichtung in Grad

In [198]:
wind = DWDWind(stgt_stats_df["Stations_id"].values)
wind_df = join_station_data(wind.download_products(), stgt_stats_df)

In [220]:
wind_df.sample(10)

,STATIONS_ID,MESS_DATUM,QN_3,F,D,TIMESTAMP,Stationshoehe,geoBreite,geoLaenge,Stationsname
690401,4928,2010012617,10,3.0,30,2010-01-26 17:00:00,314,48.8282,9.2000,Stuttgart (Schnarrenberg)
249562,4927,1967082309,5,0.5,-999,1967-08-23 09:00:00,286,48.7693,9.1814,Stuttgart-Stadt
279714,4927,1971013017,5,0.0,-999,1971-01-30 17:00:00,286,48.7693,9.1814,Stuttgart-Stadt
968214,4931,1977041403,5,2.1,250,1977-04-14 03:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
857846,4931,1963091011,5,1.6,-999,1963-09-10 11:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
1082794,4931,1990051007,10,0.9,130,1990-05-10 07:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
705980,4928,2011110620,10,2.2,290,2011-11-06 20:00:00,314,48.8282,9.2000,Stuttgart (Schnarrenberg)
1227892,4931,2006120807,10,1.2,90,2006-12-08 07:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
1049516,4931,1986072317,10,6.4,240,1986-07-23 17:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
1082756,4931,1990050817,10,2.1,260,1990-05-08 17:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen


In [3]:
sess = SparkSession.builder \
                   .master("local[*]") \
                   .config("spark.driver.memory", "32g") \
                   .getOrCreate()

In [211]:
def sanitize_column_name(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

def sanitize_all_columns(df):
    for orig_col in df.columns:
        df = df.withColumnRenamed(orig_col, sanitize_column_name(orig_col))
    return df

In [213]:
sanitize_all_columns(sess.createDataFrame(temp_df)).write.mode("overwrite").parquet("./dwd/temp.parquet")
sanitize_all_columns(sess.createDataFrame(pert_df)).write.mode("overwrite").parquet("./dwd/precipitation.parquet")
sanitize_all_columns(sess.createDataFrame(cloud_df)).write.mode("overwrite").parquet("./dwd/cloudiness.parquet")
sanitize_all_columns(sess.createDataFrame(sun_df)).write.mode("overwrite").parquet("./dwd/sun.parquet")
sanitize_all_columns(sess.createDataFrame(wind_df)).write.mode("overwrite").parquet("./dwd/wind.parquet")

In [7]:
wind_df = sess.read.parquet('./wind.parquet')

In [9]:
wind_df.show()

+-----------+----------+----+---+---+------------------+-------------+---------+---------+--------------------+
|STATIONS_ID|MESS_DATUM|QN_3|  F|  D|         TIMESTAMP|Stationshoehe|geoBreite|geoLaenge|        Stationsname|
+-----------+----------+----+---+---+------------------+-------------+---------+---------+--------------------+
|       4927|1979021423|   5|4.3|210|287881200000000000|          286|  48.7693|   9.1814|Stuttgart-Stadt  ...|
|       4927|1979021500|   5|3.3|200|287884800000000000|          286|  48.7693|   9.1814|Stuttgart-Stadt  ...|
|       4927|1979021501|   5|2.5|210|287888400000000000|          286|  48.7693|   9.1814|Stuttgart-Stadt  ...|
|       4927|1979021502|   5|2.7|200|287892000000000000|          286|  48.7693|   9.1814|Stuttgart-Stadt  ...|
|       4927|1979021503|   5|3.7|200|287895600000000000|          286|  48.7693|   9.1814|Stuttgart-Stadt  ...|
|       4927|1979021504|   5|2.8|200|287899200000000000|          286|  48.7693|   9.1814|Stuttgart-Stad